<a href="https://colab.research.google.com/github/whykusanagi/vThree_Sizes/blob/main/Vtuber_3_Sizes_Estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# vThree_Sizes Character Measurement Calculation
Calculator for vtubers to determine their 3 sizes accurately.
This notebook processes an anime character image to extract height, bust, waist, underbust, and hip measurements using Mediapipe Pose.

Launch in google collab and upload your images directly in `/content/` for easy analysis.

### Conversion Explanation
- The reference character height is set to **175 cm**. As my character is tall and based on the male vtuber model she is derived from.
- The **Nude** height in pixels (1410 pixels) is used as the baseline for conversion.
- The **cm/pixel ratio** is calculated as:
  
  \[
  cm\_per\_pixel = \frac{175}{1410} \approx 0.1241 \text{ cm/pixel}
  \]

You can change the calculation as needed, but if your character is smaller you can adjust the parameter to the average height.
The average American woman is approximately 5 feet 4 inches (162.5 cm) according to the CDC.

### Cup Size Calculation
- Cup size is determined by the difference between the bust and underbust measurements, following standard thresholds:
  - **AA**: 1.0-2.5 cm
  - **A**: 2.6-3.5 cm
  - **B**: 3.6-5.0 cm
  - **C**: 5.1-6.5 cm
  - **D**: 6.6-8.0 cm
  - **DD/E**: 8.1-9.5 cm
  - **F**: 9.6-11.0 cm
  - **G**: 11.1-12.5 cm
  - **H**: 12.6-14.0 cm

### Output Data Table
The final table provides measurements in centimeters for each outfit variation, including the estimated cup size.


If you found this tool helpful, donate so I can spend more time building random projects like this for the community.
![Donate](https://s3.whykusanagi.xyz/Simple_Header_Image.png)(https://streamelements.com/whykusanagi/tip)

In [ ]:
!pip install mediapipe

In [4]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
import pandas as pd

In [16]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
import pandas as pd
import os

# Path to images folder
image_folder = "/content/"

# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=True, model_complexity=2)

# Function to process image and extract keypoints
def process_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    if results.pose_landmarks:
        annotated_image = image.copy()
        mp_drawing.draw_landmarks(
            annotated_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        return results, image
    else:
        print(f"No pose detected in {image_path}")
        return None, image

# Function to extract keypoints and estimate circumferences
def get_body_measurements(results, image_shape):
    if not results or not results.pose_landmarks:
        return None

    h, w, _ = image_shape
    keypoints = {i: (int(lm.x * w), int(lm.y * h)) for i, lm in enumerate(results.pose_landmarks.landmark)}

    height = abs(keypoints[mp_pose.PoseLandmark.NOSE][1] - keypoints[mp_pose.PoseLandmark.LEFT_ANKLE][1])
    bust_width = abs(keypoints[mp_pose.PoseLandmark.LEFT_SHOULDER][0] - keypoints[mp_pose.PoseLandmark.RIGHT_SHOULDER][0])
    waist_width = abs(keypoints[mp_pose.PoseLandmark.LEFT_HIP][0] - keypoints[mp_pose.PoseLandmark.RIGHT_HIP][0])
    underbust_width = abs(keypoints[mp_pose.PoseLandmark.LEFT_RIB][0] - keypoints[mp_pose.PoseLandmark.RIGHT_RIB][0]) if hasattr(mp_pose.PoseLandmark, 'LEFT_RIB') else waist_width - 5
    hip_width = abs(keypoints[mp_pose.PoseLandmark.LEFT_HIP][0] - keypoints[mp_pose.PoseLandmark.RIGHT_HIP][0])

    bust_circumference = bust_width * np.pi / 2
    waist_circumference = waist_width * np.pi / 2
    underbust_circumference = underbust_width * np.pi / 2
    hip_circumference = hip_width * np.pi / 2

    return {
        "Height": height,
        "Bust Circumference": bust_circumference,
        "Waist Circumference": waist_circumference,
        "Underbust Circumference": underbust_circumference,
        "Hip Circumference": hip_circumference,
    }

# Process all images in folder
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
all_measurements = {}

for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    results, image = process_image(image_path)
    measurements = get_body_measurements(results, image.shape)

    if measurements:
        converted_measurements = {key: round(value * (175 / 1410), 1) for key, value in measurements.items()}

        def refine_estimate_cup_size(bust, underbust):
            bust_difference = bust - underbust
            refined_cup_sizes = {
                (1.0, 2.5): "AA",
                (2.6, 3.5): "A",
                (3.6, 5.0): "B",
                (5.1, 6.5): "C",
                (6.6, 8.0): "D",
                (8.1, 9.5): "DD/E",
                (9.6, 11.0): "F",
                (11.1, 12.5): "G",
                (12.6, 14.0): "H",
                (14.1, 16.0): "I",
                (16.1, 18.0): "J",
                (18.1, 20.0): "K",
                (20.1, 22.0): "L",
                (22.1, 24.0): "M",
                (24.1, 26.0): "N",
                (26.1, 28.0): "O"
            }
            for (lower, upper), size in refined_cup_sizes.items():
                if lower <= bust_difference <= upper:
                    return size
            return "Unknown"

        cup_size = refine_estimate_cup_size(converted_measurements["Bust Circumference"], converted_measurements["Underbust Circumference"])
        converted_measurements["Cup Size"] = cup_size
        all_measurements[image_file] = converted_measurements

# Display results
df = pd.DataFrame.from_dict(all_measurements, orient='index')
print(df)

                  Height  Bust Circumference  Waist Circumference  \
no_dress.png       137.4                49.5                 32.9   
marie_dress.png    175.0                45.8                 32.8   
nude_body.png      175.0                48.2                 32.9   
plug_suit.png      177.9                47.0                 34.5   
bunny_outfit.png   179.8                50.9                 35.1   

                  Underbust Circumference  Hip Circumference Cup Size  
no_dress.png                         32.0               32.9        J  
marie_dress.png                      31.8               32.8        H  
nude_body.png                        32.0               32.9        J  
plug_suit.png                        33.5               34.5        H  
bunny_outfit.png                     34.1               35.1        J  
